In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(""),".."))

import custom
from tqdm import tqdm
import numpy
import pickle
import pandas
from IPython.display import display

import torch
import torch.nn as nn
from torch.utils.data import DataLoader


In [2]:
with open("data/korean_word.pkl", mode = "rb") as f :
    word_dict = pickle.load(f)

with open("data/korean_vector_list.pkl", mode = "rb") as f :
    vec_list = pickle.load(f)

train_df = pandas.read_csv("data/korean_movie_train.txt", sep = "\t", encoding = "UTF8", index_col = 0)
test_df = pandas.read_csv("data/korean_movie_test.txt", sep = "\t", encoding = "UTF8", index_col = 0)

print(len(word_dict))
print(len(vec_list))
display(train_df)
display(test_df)

30188
30188


,document,label,word_len
0,아 더빙 . 진짜 짜증 나 네요 목소리,0,8
1,흠 . 포스터 보고 초딩 영화 줄 . 오버 연기 조차 가볍 지 않 구나,1,15
2,너무재밓 었 다 그래서 보 는 것 을 추천 하 ᆫ다,0,11
3,교도소 이야기 이 구먼 . 솔직히 재미 는 없 다 . 평점 조정,0,13
4,사이몬페그 의 익살 스럽 ᆫ 연기 가 돋보이 었 던 영화 ! 스파이더맨 에서 늙 어...,1,32
...,...,...,...
149995,인간 이 문제 이 지 . 소 는 뭔 죄 이 ᆫ가 .,0,13
149996,평점 이 너무 낮 어서 .,1,6
149997,이것 이 뭐 요 ? 한국인 은 거들먹거리 고 필리핀 혼혈 은 착하 다 ?,0,15
149998,청춘 영화 의 최 고봉 . 방황 과 우울 하 었 던 날 들 의 자화상,1,16


,document,label,word_len
0,굳,1,1
2,뭐 이 야 이 평점 들 은 . 나쁘 지 ᆫ 않 지만 점 짜리 는 더더욱 아니 잖아,0,19
3,지루하 지 는 않 은데 완전 막장 이 ᆷ . 돈 주 고 보 기 에 는 .,0,18
4,만 아니 었 어도 별 다섯 개 주 었 을 터 이 ᆫ데 . 왜 로 나오 어서 저 의 ...,0,28
5,음악 이 주 가 되 ᆫ 최고 의 음악 영화,1,10
...,...,...,...
49995,오랜만 에 평점 로 이 기 ᆫ 하 었 네 킹왕짱 쌈 뽕 하 ᆫ 영화 를 만나 었 습...,1,25
49996,의지 박약 들 이나 하 는 거 다 탈영 은 일단 주인공 김대희 닮 었 고 이등병 찐따,0,18
49997,그림 도 좋 고 완성도 도 높 었 지만 . 보 는 내내 불안 하 게 만들 ᆫ다,0,18
49998,절대 보 어서 는 안 되 ᆯ 영화 . 재미 도 없 고 기분 만 잡치 고 . 한 세트...,0,26


In [3]:
x = []
for i in tqdm(train_df.index) :
    x.append(custom.word_vectorize(train_df.loc[i, "document"], word_dict, 30))

t = train_df['label'].values.tolist()

print(len(x[1]))

device = "cuda" if torch.cuda.is_available() else "cpu"

tensor_x = torch.tensor(x, dtype = torch.long, device = device)
tensor_t = torch.tensor(t, dtype = torch.long, device = device)

dataloader = DataLoader(list(zip(tensor_x, tensor_t)),batch_size=300,shuffle=True)


print()
print(tensor_x.shape)
print(tensor_t.shape)

100%|███████████████████████████████████████████████████████████████████████| 148963/148963 [00:06<00:00, 23762.16it/s]


30

torch.Size([148963, 30])
torch.Size([148963])


In [4]:
x = []
for i in tqdm(test_df.index) :
    x.append(custom.word_vectorize(test_df.loc[i, "document"], word_dict, 40))

t = test_df['label'].values.tolist()



device = "cuda" if torch.cuda.is_available() else "cpu"

tensor_x = torch.tensor(x, dtype = torch.long, device = device)
tensor_t = torch.tensor(t, dtype = torch.long, device = device)

test_dataloader = DataLoader(list(zip(tensor_x, tensor_t)),batch_size=1000,shuffle=True)

print()
print(len(tensor_x.shape))
print(len(tensor_t.shape))

100%|█████████████████████████████████████████████████████████████████████████| 49622/49622 [00:02<00:00, 21938.43it/s]



2
1


In [5]:
##AI
class NN(nn.Module) :
    def __init__(self, embedding_tensor) :
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_tensor, freeze=True, padding_idx=0)
        self.rnn = nn.LSTM(embedding_tensor.shape[1],10,batch_first = True)
        self.f = nn.Sequential(
            nn.Linear(10,2),
        )
    def forward(self, x) :
        x = self.embedding(x)
        x, h = self.rnn(x)
        x = x[:,-1,:]
        x = self.f(x)
        return x

embedding_tensor = torch.tensor(vec_list, dtype = torch.float)
F = NN(embedding_tensor)
F = F.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(F.parameters(), lr = 2)
epoch = 30
prev_acc = 0
cnt = 0

for e in range(epoch) :
    loss_sum = 0
    for x, t in dataloader :
        #
        y = F(x)
        #
        loss = loss_function(y, t)
        loss_sum += loss.item()
        #
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_sum /= len(dataloader)

    #
    correct = 0
    total = 0
    for x, t in test_dataloader :
        with torch.no_grad() :
            y = F(x)
        correct += (y.argmax(dim = -1) == t).sum().item()
        total += len(x)
    acc = correct / total

    if acc <= prev_acc + 0.001 :
        cnt += 1
    else :
        cnt = 0
        prev_acc = acc
    print(f"epoch {e+1} | loss {loss_sum} | acc {acc} | cnt {cnt}")
    if cnt >= 5 :
        print("train end")
        break



epoch 1 | loss 0.6108079397942219 | acc 0.7500302285276692 | cnt 0
epoch 2 | loss 0.4821653300727637 | acc 0.7820523155052195 | cnt 0
epoch 3 | loss 0.4583440205940538 | acc 0.7686106968683245 | cnt 1
epoch 4 | loss 0.4436380791832024 | acc 0.7714118737656684 | cnt 2
epoch 5 | loss 0.431205969881004 | acc 0.7933779372052718 | cnt 0
epoch 6 | loss 0.4214150925997038 | acc 0.8043609689250736 | cnt 0
epoch 7 | loss 0.413586242398026 | acc 0.8006730885494338 | cnt 1
epoch 8 | loss 0.40538035426101454 | acc 0.8128451090242231 | cnt 0
epoch 9 | loss 0.3999943653462638 | acc 0.8002095844585063 | cnt 1
epoch 10 | loss 0.3936622620108622 | acc 0.7862036999717867 | cnt 2
epoch 11 | loss 0.390005303400623 | acc 0.7907178267703842 | cnt 3
epoch 12 | loss 0.3859336516506955 | acc 0.8010358308814639 | cnt 4
epoch 13 | loss 0.38139531825869616 | acc 0.8193946233525452 | cnt 0
epoch 14 | loss 0.37839945365245675 | acc 0.8220345814356536 | cnt 0
epoch 15 | loss 0.3750091276658133 | acc 0.82352585546733

In [6]:
torch.save(F.to("cpu"), "korean_movie.pt")